# 🎵 DTLN Multi-Method Audio Evaluation
**Evaluation and Comparison of Three Audio Processing Methods**

Interactive notebook for noise suppression experiments with DTLN ONNX model and traditional DSP methods.

## 🎯 Main Features:

### Processing Methods:
- ✅ **Deterministic**: Fixed noise + DTLN (reproducible)
- ✅ **Stochastic**: Random noise + DTLN (different each run)  
- ✅ **Traditional**: Spectral Subtraction & Wiener Filter

### Notebook Features:
- 📁 Automatic folder structure creation
- 🎨 2x2 spectrogram visualization with magma colormap
- 📊 Complete metric evaluation (STOI, PESQ, MSE, MRE) - 4 decimals
- 🔄 Automatic multi-condition batch processing
- 📊 Excel export with 3 sheets
- 🎵 Interactive audio player

**Quick Start:**
1. Install packages → 2. Upload models → 3. Upload audio → 4. Run experiments → 5. Download results

In [ ]:
# Install Required Packages
!pip install -q numpy scipy librosa soundfile onnxruntime matplotlib pystoi pesq openpyxl pandas

import numpy as np
import soundfile as sf
import librosa
import librosa.display
import matplotlib.pyplot as plt
import onnxruntime
from scipy import signal
from pystoi import stoi
from pesq import pesq
import os
import shutil
from google.colab import files
from IPython.display import display, Audio, HTML, clear_output
import time
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

# Setup directories (same as Flask app)
os.makedirs('pretrained_model', exist_ok=True)
os.makedirs('uploads', exist_ok=True)
os.makedirs('outputs', exist_ok=True)
os.makedirs('static/spectrograms', exist_ok=True)
os.makedirs('results/audio', exist_ok=True)
os.makedirs('results/spectrograms', exist_ok=True)
os.makedirs('results/metrics', exist_ok=True)

print("✅ All packages installed and directories created!")
print("📁 Directory structure matches Flask app (app.py)")

## 📦 Setup Models and Audio

In [ ]:
# Upload DTLN Models
print("📤 Upload model_1.onnx and model_2.onnx:")
uploaded = files.upload()
for filename in uploaded.keys():
    shutil.move(filename, f'pretrained_model/{filename}')
    print(f'✓ {filename} uploaded to pretrained_model/')

if os.path.exists('pretrained_model/model_1.onnx') and os.path.exists('pretrained_model/model_2.onnx'):
    print("\n✅ Both DTLN models ready in pretrained_model/!")
    
    # Load models with configuration matching app.py
    BLOCK_LEN = 512
    BLOCK_SHIFT = 128
    SAMPLE_RATE = 16000
    
    interpreter_1 = onnxruntime.InferenceSession('pretrained_model/model_1.onnx')
    interpreter_2 = onnxruntime.InferenceSession('pretrained_model/model_2.onnx')
    model_input_names_1 = [inp.name for inp in interpreter_1.get_inputs()]
    model_input_names_2 = [inp.name for inp in interpreter_2.get_inputs()]
    model_inputs_1 = {inp.name: np.zeros([dim if isinstance(dim, int) else 1 for dim in inp.shape], dtype=np.float32) for inp in interpreter_1.get_inputs()}
    model_inputs_2 = {inp.name: np.zeros([dim if isinstance(dim, int) else 1 for dim in inp.shape], dtype=np.float32) for inp in interpreter_2.get_inputs()}
    
    print(f"✓ Models loaded: BLOCK_LEN={BLOCK_LEN}, BLOCK_SHIFT={BLOCK_SHIFT}, SAMPLE_RATE={SAMPLE_RATE}Hz")
    print("✓ Configuration matches app.py")
else:
    print("\n⚠️ Please upload both model files!")

In [ ]:
# Upload Audio Files
print("📤 Upload Clean Audio (required):")
uploaded_clean = files.upload()
audio_clean_file = list(uploaded_clean.keys())[0]
shutil.move(audio_clean_file, f'uploads/{audio_clean_file}')
print(f"✓ {audio_clean_file} → uploads/")

print("\n📤 Upload Noise Audio (optional, for deterministic method):")
uploaded_noise = files.upload()
audio_noise_file = list(uploaded_noise.keys())[0] if uploaded_noise else None
if audio_noise_file:
    shutil.move(audio_noise_file, f'uploads/{audio_noise_file}')
    print(f"✓ {audio_noise_file} → uploads/")
else:
    print("✓ Skip - will use synthetic noise for stochastic")

# Load and preprocess (same as app.py)
def load_audio(path, sr=16000):
    """Load audio file and resample if necessary"""
    audio, orig_sr = sf.read(path)
    
    # Convert to mono if stereo
    if len(audio.shape) > 1:
        audio = np.mean(audio, axis=1)
    
    # Resample if necessary
    if orig_sr != sr:
        num_samples = int(len(audio) * sr / orig_sr)
        audio = signal.resample(audio, num_samples)
    
    return audio, sr

audio_clean, sr = load_audio(f'uploads/{audio_clean_file}', SAMPLE_RATE)
audio_noise_uploaded = load_audio(f'uploads/{audio_noise_file}', SAMPLE_RATE)[0] if audio_noise_file else None

print(f"\n✅ Clean: {len(audio_clean)/sr:.2f}s @ {sr}Hz")
if audio_noise_uploaded is not None:
    print(f"✅ Noise: {len(audio_noise_uploaded)/sr:.2f}s (uploaded)")
else:
    print("✅ Noise: Will use synthetic (Gaussian/White/Mixed)")

print("\n🎵 Audio Preview:")
display(Audio(audio_clean, rate=sr))

## ⚙️ Configuration & Processing Auto-Run
**Automatically generate, process, and visualize when parameters are changed**

Select method and configuration below, then run to see results.

In [ ]:
#@title 🎛️ Processing Configuration { run: "auto" }

#@markdown ---
#@markdown ### **Select Processing Method:**
method = "deterministic"  #@param ["deterministic", "stochastic", "traditional"]

#@markdown ---
#@markdown ### **DETERMINISTIC Settings** (Fixed noise + DTLN)
#@markdown Upload noise required, SNR is fixed
snr_deterministic = 10  #@param {type:"number"}

#@markdown ---
#@markdown ### **STOCHASTIC Settings** (Random synthetic noise + DTLN)
#@markdown Choose noise type, no upload needed
noise_type_stochastic = "gaussian"  #@param ["gaussian", "white", "random_snr", "mixed"]
snr_stochastic = 10  #@param {type:"number"}

#@markdown ---
#@markdown ### **TRADITIONAL Settings** (DSP methods)
#@markdown Upload noise required, choose DSP method
traditional_method = "spectral_subtraction"  #@param ["spectral_subtraction", "wiener"]
snr_traditional = 10  #@param {type:"number"}

print("="*80)
print(f"🎯 PROCESSING CONFIGURATION")
print("="*80)
print(f"Method Selected: {method.upper()}")
print()

# ============== NOISE GENERATION (same as app.py) ==============
def add_gaussian_noise(audio, snr_db):
    """Add Gaussian random noise to audio"""
    audio_power = np.mean(audio ** 2)
    noise_power = audio_power / (10 ** (snr_db / 10))
    noise = np.random.normal(0, np.sqrt(noise_power), len(audio))
    return audio + noise, noise

def add_white_noise(audio, snr_db):
    """Add white noise to audio"""
    audio_power = np.mean(audio ** 2)
    noise_power = audio_power / (10 ** (snr_db / 10))
    noise = np.random.uniform(-1, 1, len(audio)) * np.sqrt(noise_power * 3)
    return audio + noise, noise

def mix_audio_with_snr(clean, noise, snr_db):
    """Mix clean audio with noise at specified SNR"""
    if len(noise) < len(clean):
        repeats = int(np.ceil(len(clean) / len(noise)))
        noise = np.tile(noise, repeats)[:len(clean)]
    else:
        noise = noise[:len(clean)]
    
    clean_power = np.mean(clean ** 2)
    noise_power = np.mean(noise ** 2)
    
    if noise_power == 0:
        noise_power = 1e-10
    
    scale = np.sqrt(clean_power / (noise_power * (10 ** (snr_db / 10))))
    scaled_noise = noise * scale
    mixed = clean + scaled_noise
    
    max_val = np.max(np.abs(mixed))
    if max_val > 1.0:
        mixed = mixed / max_val
        scaled_noise = scaled_noise / max_val
    
    return mixed, scaled_noise

# ============== PROCESS BASED ON METHOD (exact match to app.py) ==============
if method == 'deterministic':
    # Deterministic: Use uploaded noise file with fixed SNR
    if audio_noise_uploaded is None:
        print("❌ ERROR: Deterministic method requires noise audio file!")
        print("   Please upload noise audio in the previous cell.")
        raise ValueError("Deterministic method requires noise audio file")
    
    snr_db = snr_deterministic
    mixed_audio, used_noise = mix_audio_with_snr(audio_clean, audio_noise_uploaded, snr_db)
    method_description = f"Deterministic (Fixed Noise, SNR={snr_db}dB)"
    
    print(f"✅ Configuration:")
    print(f"   - Noise: Uploaded file (fixed)")
    print(f"   - SNR: {snr_db} dB")
    print(f"   - Processing: DTLN neural network")
    
elif method == 'stochastic':
    # Stochastic: Random noise types (no upload needed)
    snr_db = snr_stochastic
    
    if noise_type_stochastic == 'gaussian':
        mixed_audio, used_noise = add_gaussian_noise(audio_clean, snr_db)
        noise_desc = f"Gaussian Noise (SNR={snr_db}dB)"
    elif noise_type_stochastic == 'white':
        mixed_audio, used_noise = add_white_noise(audio_clean, snr_db)
        noise_desc = f"White Noise (SNR={snr_db}dB)"
    elif noise_type_stochastic == 'random_snr':
        # Random SNR between -5 and 20 dB
        random_snr = np.random.uniform(-5, 20)
        mixed_audio, used_noise = add_gaussian_noise(audio_clean, random_snr)
        noise_desc = f"Random SNR Gaussian ({random_snr:.1f}dB)"
        snr_db = random_snr
    elif noise_type_stochastic == 'mixed':
        # Mix of different noise types
        noise_choice = np.random.choice(['gaussian', 'white'])
        random_snr = np.random.uniform(0, 15)
        if noise_choice == 'gaussian':
            mixed_audio, used_noise = add_gaussian_noise(audio_clean, random_snr)
        else:
            mixed_audio, used_noise = add_white_noise(audio_clean, random_snr)
        noise_desc = f"Mixed Random ({noise_choice.title()}, {random_snr:.1f}dB)"
        snr_db = random_snr
    else:
        mixed_audio, used_noise = add_gaussian_noise(audio_clean, snr_db)
        noise_desc = f"Gaussian Noise (SNR={snr_db}dB)"
    
    method_description = f"Stochastic ({noise_desc})"
    
    print(f"✅ Configuration:")
    print(f"   - Noise: {noise_desc}")
    print(f"   - Processing: DTLN neural network")
    
elif method == 'traditional':
    # Traditional: DSP methods (upload noise or use synthetic)
    snr_db = snr_traditional
    
    if audio_noise_uploaded is not None:
        mixed_audio, used_noise = mix_audio_with_snr(audio_clean, audio_noise_uploaded, snr_db)
        noise_source = "Uploaded file"
    else:
        # Use synthetic noise
        mixed_audio, used_noise = add_gaussian_noise(audio_clean, snr_db)
        noise_source = "Synthetic Gaussian"
    
    # Apply traditional processing
    if traditional_method == 'spectral_subtraction':
        trad_method_name = "Spectral Subtraction"
    elif traditional_method == 'wiener':
        trad_method_name = "Wiener Filter"
    else:
        trad_method_name = "Spectral Subtraction"
    
    method_description = f"Traditional ({trad_method_name}, SNR={snr_db}dB)"
    
    print(f"✅ Configuration:")
    print(f"   - Noise: {noise_source}")
    print(f"   - SNR: {snr_db} dB")
    print(f"   - Processing: {trad_method_name} (DSP)")

print()
print("🎵 Noisy Audio Preview:")
display(Audio(mixed_audio, rate=SAMPLE_RATE))

# ============== PROCESSING (same as app.py) ==============
def process_dtln(audio):
    """Process audio using DTLN ONNX models"""
    # Reset model states
    for inp in interpreter_1.get_inputs():
        model_inputs_1[inp.name] = np.zeros([dim if isinstance(dim, int) else 1 for dim in inp.shape], dtype=np.float32)
    for inp in interpreter_2.get_inputs():
        model_inputs_2[inp.name] = np.zeros([dim if isinstance(dim, int) else 1 for dim in inp.shape], dtype=np.float32)
    
    out_file = np.zeros(len(audio))
    in_buffer = np.zeros(BLOCK_LEN, dtype='float32')
    out_buffer = np.zeros(BLOCK_LEN, dtype='float32')
    num_blocks = (len(audio) - (BLOCK_LEN - BLOCK_SHIFT)) // BLOCK_SHIFT
    
    for idx in range(num_blocks):
        in_buffer[:-BLOCK_SHIFT] = in_buffer[BLOCK_SHIFT:]
        in_buffer[-BLOCK_SHIFT:] = audio[idx*BLOCK_SHIFT:(idx*BLOCK_SHIFT)+BLOCK_SHIFT]
        
        in_fft = np.fft.rfft(in_buffer)
        in_mag, in_phase = np.abs(in_fft), np.angle(in_fft)
        
        model_inputs_1[model_input_names_1[0]] = np.reshape(in_mag, (1,1,-1)).astype('float32')
        out_1 = interpreter_1.run(None, model_inputs_1)
        model_inputs_1[model_input_names_1[1]] = out_1[1]
        
        est_complex = in_mag * out_1[0] * np.exp(1j * in_phase)
        est_block = np.fft.irfft(est_complex)
        
        model_inputs_2[model_input_names_2[0]] = np.reshape(est_block, (1,1,-1)).astype('float32')
        out_2 = interpreter_2.run(None, model_inputs_2)
        model_inputs_2[model_input_names_2[1]] = out_2[1]
        
        out_buffer[:-BLOCK_SHIFT] = out_buffer[BLOCK_SHIFT:]
        out_buffer[-BLOCK_SHIFT:] = 0
        out_buffer += np.squeeze(out_2[0])
        out_file[idx*BLOCK_SHIFT:(idx*BLOCK_SHIFT)+BLOCK_SHIFT] = out_buffer[:BLOCK_SHIFT]
    return out_file

def spectral_subtraction(noisy_audio, noise_profile=None, alpha=2.0):
    """Traditional spectral subtraction method"""
    nperseg = 512
    noverlap = 384
    
    f, t, Zxx = signal.stft(noisy_audio, fs=SAMPLE_RATE, nperseg=nperseg, noverlap=noverlap)
    
    if noise_profile is None:
        noise_frames = int(0.1 * Zxx.shape[1])
        noise_spectrum = np.mean(np.abs(Zxx[:, :noise_frames]) ** 2, axis=1, keepdims=True)
    else:
        _, _, noise_stft = signal.stft(noise_profile, fs=SAMPLE_RATE, nperseg=nperseg, noverlap=noverlap)
        noise_spectrum = np.mean(np.abs(noise_stft) ** 2, axis=1, keepdims=True)
    
    magnitude = np.abs(Zxx)
    phase = np.angle(Zxx)
    clean_magnitude = np.sqrt(np.maximum(magnitude ** 2 - alpha * noise_spectrum, 0.01 * magnitude ** 2))
    clean_stft = clean_magnitude * np.exp(1j * phase)
    _, clean_audio = signal.istft(clean_stft, fs=SAMPLE_RATE, nperseg=nperseg, noverlap=noverlap)
    
    if len(clean_audio) > len(noisy_audio):
        clean_audio = clean_audio[:len(noisy_audio)]
    elif len(clean_audio) < len(noisy_audio):
        clean_audio = np.pad(clean_audio, (0, len(noisy_audio) - len(clean_audio)))
    
    return clean_audio

def wiener_filter(noisy_audio, noise_profile=None):
    """Traditional Wiener filtering method"""
    nperseg = 512
    noverlap = 384
    
    f, t, Zxx = signal.stft(noisy_audio, fs=SAMPLE_RATE, nperseg=nperseg, noverlap=noverlap)
    
    if noise_profile is None:
        noise_frames = int(0.1 * Zxx.shape[1])
        noise_power = np.mean(np.abs(Zxx[:, :noise_frames]) ** 2, axis=1, keepdims=True)
    else:
        _, _, noise_stft = signal.stft(noise_profile, fs=SAMPLE_RATE, nperseg=nperseg, noverlap=noverlap)
        noise_power = np.mean(np.abs(noise_stft) ** 2, axis=1, keepdims=True)
    
    noisy_power = np.abs(Zxx) ** 2
    wiener_gain = np.maximum(1 - noise_power / (noisy_power + 1e-10), 0.1)
    clean_stft = Zxx * wiener_gain
    _, clean_audio = signal.istft(clean_stft, fs=SAMPLE_RATE, nperseg=nperseg, noverlap=noverlap)
    
    if len(clean_audio) > len(noisy_audio):
        clean_audio = clean_audio[:len(noisy_audio)]
    elif len(clean_audio) < len(noisy_audio):
        clean_audio = np.pad(clean_audio, (0, len(noisy_audio) - len(clean_audio)))
    
    return clean_audio

# Process audio based on method
print("\n⚙️  Processing audio...")
start = time.time()

if method in ["deterministic", "stochastic"]:
    # Process with DTLN
    audio_processed = process_dtln(mixed_audio)
    
elif method == "traditional":
    # Apply traditional processing
    if traditional_method == 'spectral_subtraction':
        audio_processed = spectral_subtraction(mixed_audio, used_noise)
    else:  # wiener
        audio_processed = wiener_filter(mixed_audio, used_noise)
    
    # Match length
    if len(audio_processed) != len(audio_clean):
        audio_processed = audio_processed[:len(audio_clean)] if len(audio_processed) > len(audio_clean) else np.pad(audio_processed, (0, len(audio_clean)-len(audio_processed)))

proc_time = time.time() - start

# ============== METRICS (4 decimal precision - same as app.py) ==============
def calculate_metrics(clean, processed, fs=16000):
    """Calculate evaluation metrics: STOI, PESQ, MSE, MRE"""
    metrics = {}
    
    min_len = min(len(clean), len(processed))
    clean = clean[:min_len]
    processed = processed[:min_len]
    
    try:
        metrics['stoi'] = stoi(clean, processed, fs, extended=False)
    except Exception as e:
        metrics['stoi'] = 0.0
    
    try:
        if fs == 16000:
            metrics['pesq'] = pesq(fs, clean, processed, 'wb')
        elif fs == 8000:
            metrics['pesq'] = pesq(fs, clean, processed, 'nb')
        else:
            metrics['pesq'] = 0.0
    except Exception as e:
        metrics['pesq'] = 0.0
    
    metrics['mse'] = np.mean((clean - processed) ** 2)
    epsilon = 1e-10
    metrics['mre'] = np.mean(np.abs(clean - processed) / (np.abs(clean) + epsilon))
    
    return metrics

metrics = calculate_metrics(audio_clean, audio_processed, SAMPLE_RATE)

# Save outputs (same structure as app.py)
fname = f"{method}_{snr_db:.0f}dB"
sf.write(f'outputs/{fname}_mixed.wav', mixed_audio, SAMPLE_RATE)
sf.write(f'outputs/{fname}_processed.wav', audio_processed, SAMPLE_RATE)

print(f"✅ Processing completed in {proc_time:.3f}s")
print(f"   Realtime factor: {proc_time/(len(audio_clean)/SAMPLE_RATE):.2f}x")
print()
print("🎵 Processed Audio Preview:")
display(Audio(audio_processed, rate=SAMPLE_RATE))

# ============== DISPLAY METRICS (4 decimal - same as app.py) ==============
print(f"\n{'='*85}")
print(f"📊 EVALUATION METRICS - {method_description}")
print(f"{'='*85}")
print(f"{'Metric':<15} {'Value':<20} {'Quality'}")
print(f"{'-'*85}")
print(f"{'STOI':<15} {metrics['stoi']:<20.4f} {'🟢 Excellent' if metrics['stoi'] > 0.8 else '🟡 Good' if metrics['stoi'] > 0.6 else '🔴 Poor'}")
print(f"{'PESQ':<15} {metrics['pesq']:<20.4f} {'🟢 Excellent' if metrics['pesq'] > 3.5 else '🟡 Good' if metrics['pesq'] > 2.5 else '🔴 Poor'}")
print(f"{'MSE':<15} {metrics['mse']:<20.4f} {'(lower is better)'}")
print(f"{'MRE':<15} {metrics['mre']:<20.4f} {'(lower is better)'}")
print(f"{'='*85}")
print(f"\n💡 Guide: STOI: 0-1 (>0.8=Excellent) | PESQ: 1-4.5 (>3.5=Excellent)")

# ============== VISUALIZATION (2x2 combined spectrogram with magma - same as app.py) ==============
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle(f'{method_description}', fontsize=16, fontweight='bold')

for idx, (audio, title) in enumerate([(audio_clean, 'Clean Audio'), 
                                       (used_noise, 'Noise'), 
                                       (mixed_audio, 'Mixed Audio (Noisy)'), 
                                       (audio_processed, 'Processed Audio (Denoised)')]):
    ax = axes[idx//2, idx%2]
    D = librosa.amplitude_to_db(np.abs(librosa.stft(audio)), ref=np.max)
    img = librosa.display.specshow(D, sr=SAMPLE_RATE, x_axis='time', y_axis='hz', ax=ax, cmap='magma')
    ax.set_title(title, fontweight='bold', fontsize=12)
    plt.colorbar(img, ax=ax, format='%+2.0f dB')

plt.tight_layout()
plt.savefig(f'static/spectrograms/{fname}_combined_spec.png', dpi=120, bbox_inches='tight')
plt.show()

# ============== METRICS BAR CHART ==============
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
fig.suptitle(f'Metrics - {method_description}', fontsize=14, fontweight='bold')

charts = [
    ('STOI', metrics['stoi'], [0, 1]),
    ('PESQ', metrics['pesq'], [0, 4.5]),
    ('MSE', metrics['mse'], None),
    ('MRE', metrics['mre'], None)
]

colors = ['#3b82f6', '#9333ea', '#f59e0b', '#f43f5e']

for idx, (title, val, ylim) in enumerate(charts):
    ax = axes[idx]
    bar = ax.bar([title], [val], color=colors[idx], width=0.6)
    ax.set_title(title, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    if ylim: ax.set_ylim(ylim)
    
    ax.text(0, val, f'{val:.4f}', ha='center', va='bottom', fontweight='bold', fontsize=11)

plt.tight_layout()
plt.savefig(f'results/metrics/{fname}_chart.png', dpi=120, bbox_inches='tight')
plt.show()

# Save metrics to text file (same as app.py output format)
with open(f'results/metrics/{fname}_metrics.txt', 'w') as f:
    f.write(f"DTLN Multi-Method Audio Evaluation\n")
    f.write(f"Method: {method_description}\n")
    f.write(f"SNR: {snr_db:.4f} dB\n")
    f.write("="*80+"\n\n")
    f.write(f"STOI: {metrics['stoi']:.4f}\n")
    f.write(f"PESQ: {metrics['pesq']:.4f}\n")
    f.write(f"MSE: {metrics['mse']:.4f}\n")
    f.write(f"MRE: {metrics['mre']:.4f}\n")
    f.write(f"\nProcessing Time: {proc_time:.4f}s\n")
    f.write(f"Realtime Factor: {proc_time/(len(audio_clean)/SAMPLE_RATE):.4f}x\n")

print(f"\n✅ Results saved:")
print(f"   • Audio: outputs/{fname}_mixed.wav, outputs/{fname}_processed.wav")
print(f"   • Spectrogram: static/spectrograms/{fname}_combined_spec.png")
print(f"   • Metrics: results/metrics/{fname}_metrics.txt")
print(f"   • Chart: results/metrics/{fname}_chart.png")

## 🔁 Batch Comparison Mode
Run all configurations and save to Excel

In [ ]:
#@title 🚀 Batch Processing - All Configurations { run: "auto" }
#@markdown Run comprehensive experiments for all methods and configurations
run_batch = False  #@param {type:"boolean"}

if run_batch:
    # Configuration matrix (same logic as app.py would process)
    batch_configs = []
    
    # DETERMINISTIC: requires uploaded noise, fixed SNR values
    if audio_noise_uploaded is not None:
        for snr in [0, 5, 10, 15, 20]:
            batch_configs.append({
                'method': 'deterministic',
                'snr': snr,
                'noise_type': None,
                'traditional_method': None
            })
    else:
        print("⚠️  Skipping DETERMINISTIC: No noise audio uploaded")
    
    # STOCHASTIC: 4 noise types (same as app.py), fixed SNR values
    for noise_type in ['gaussian', 'white', 'random_snr', 'mixed']:
        for snr in [0, 5, 10, 15, 20]:
            batch_configs.append({
                'method': 'stochastic',
                'snr': snr,
                'noise_type': noise_type,
                'traditional_method': None
            })
    
    # TRADITIONAL: 2 methods (same as app.py), uses uploaded noise or synthetic
    for trad_method in ['spectral_subtraction', 'wiener']:
        for snr in [0, 5, 10, 15, 20]:
            batch_configs.append({
                'method': 'traditional',
                'snr': snr,
                'noise_type': None,
                'traditional_method': trad_method
            })
    
    results_summary = []
    total = len(batch_configs)
    print(f"🔄 Running batch experiments: {total} configurations")
    print(f"   - Deterministic: {sum(1 for c in batch_configs if c['method']=='deterministic')} configs")
    print(f"   - Stochastic: {sum(1 for c in batch_configs if c['method']=='stochastic')} configs")
    print(f"   - Traditional: {sum(1 for c in batch_configs if c['method']=='traditional')} configs")
    print()
    
    for idx, config in enumerate(batch_configs, 1):
        method = config['method']
        snr_db = config['snr']
        noise_type = config['noise_type']
        trad_method = config['traditional_method']
        
        # Display progress
        progress = f"[{idx}/{total}]"
        
        # ============== PROCESS EXACT SAME AS APP.PY ==============
        try:
            if method == 'deterministic':
                # Deterministic: Use uploaded noise file with fixed SNR
                mixed_audio, used_noise = mix_audio_with_snr(audio_clean, audio_noise_uploaded, snr_db)
                method_desc = f"Deterministic (Fixed Noise, SNR={snr_db}dB)"
                noise_label = "Fixed_Noise"
                
            elif method == 'stochastic':
                # Stochastic: Random noise types (exact same as app.py)
                if noise_type == 'gaussian':
                    mixed_audio, used_noise = add_gaussian_noise(audio_clean, snr_db)
                    noise_desc = f"Gaussian Noise (SNR={snr_db}dB)"
                    noise_label = "Gaussian"
                elif noise_type == 'white':
                    mixed_audio, used_noise = add_white_noise(audio_clean, snr_db)
                    noise_desc = f"White Noise (SNR={snr_db}dB)"
                    noise_label = "White"
                elif noise_type == 'random_snr':
                    random_snr = np.random.uniform(-5, 20)
                    mixed_audio, used_noise = add_gaussian_noise(audio_clean, random_snr)
                    noise_desc = f"Random SNR Gaussian ({random_snr:.1f}dB)"
                    noise_label = f"Random_SNR_{random_snr:.1f}"
                    snr_db = random_snr  # Update for recording
                elif noise_type == 'mixed':
                    noise_choice = np.random.choice(['gaussian', 'white'])
                    random_snr = np.random.uniform(0, 15)
                    if noise_choice == 'gaussian':
                        mixed_audio, used_noise = add_gaussian_noise(audio_clean, random_snr)
                    else:
                        mixed_audio, used_noise = add_white_noise(audio_clean, random_snr)
                    noise_desc = f"Mixed Random ({noise_choice.title()}, {random_snr:.1f}dB)"
                    noise_label = f"Mixed_{noise_choice}_{random_snr:.1f}"
                    snr_db = random_snr
                else:
                    mixed_audio, used_noise = add_gaussian_noise(audio_clean, snr_db)
                    noise_desc = f"Gaussian Noise (SNR={snr_db}dB)"
                    noise_label = "Gaussian"
                
                method_desc = f"Stochastic ({noise_desc})"
                
            elif method == 'traditional':
                # Traditional: DSP methods (use uploaded noise if available, else synthetic)
                if audio_noise_uploaded is not None:
                    mixed_audio, used_noise = mix_audio_with_snr(audio_clean, audio_noise_uploaded, snr_db)
                    noise_source = "Uploaded"
                else:
                    mixed_audio, used_noise = add_gaussian_noise(audio_clean, snr_db)
                    noise_source = "Synthetic"
                
                if trad_method == 'spectral_subtraction':
                    trad_method_name = "Spectral Subtraction"
                else:  # wiener
                    trad_method_name = "Wiener Filter"
                
                method_desc = f"Traditional ({trad_method_name}, SNR={snr_db}dB)"
                noise_label = f"{trad_method_name.replace(' ', '_')}_{noise_source}"
            
            print(f"{progress} {method_desc}")
            
            # Process audio
            start = time.time()
            
            if method in ['deterministic', 'stochastic']:
                # Process with DTLN (same as app.py)
                audio_proc = process_dtln(mixed_audio)
                
            elif method == 'traditional':
                # Apply traditional processing (same as app.py)
                if trad_method == 'spectral_subtraction':
                    audio_proc = spectral_subtraction(mixed_audio, used_noise)
                else:  # wiener
                    audio_proc = wiener_filter(mixed_audio, used_noise)
                
                # Match length
                if len(audio_proc) != len(audio_clean):
                    audio_proc = audio_proc[:len(audio_clean)] if len(audio_proc) > len(audio_clean) else np.pad(audio_proc, (0, len(audio_clean)-len(audio_proc)))
            
            proc_time = time.time() - start
            
            # Calculate metrics (4 decimal precision)
            m = calculate_metrics(audio_clean, audio_proc, SAMPLE_RATE)
            
            # Save result
            results_summary.append({
                'Method': method.capitalize(),
                'Noise_Type': noise_label if method != 'traditional' else trad_method_name.replace(' ', '_'),
                'SNR_Input_dB': snr_db,
                'STOI': m['stoi'],
                'PESQ': m['pesq'],
                'MSE': m['mse'],
                'MRE': m['mre'],
                'Processing_Time_s': proc_time,
                'Realtime_Factor': proc_time / (len(audio_clean)/SAMPLE_RATE)
            })
            
            # Save audio files
            fname_batch = f"{method}_{noise_label if method != 'traditional' else trad_method.replace('_', '')}_{snr_db:.0f}dB"
            sf.write(f'results/audio/{fname_batch}_processed.wav', audio_proc, SAMPLE_RATE)
            
        except Exception as e:
            print(f"   ❌ Error: {str(e)}")
            continue
    
    # ============== EXPORT TO EXCEL (3 sheets - same as app.py would format) ==============
    df = pd.DataFrame(results_summary)
    
    # Round all numeric columns to 4 decimal places
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].round(4)
    
    excel_file = 'results/batch_results.xlsx'
    with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
        # Sheet 1: All Results (complete data)
        df.to_excel(writer, sheet_name='All_Results', index=False)
        
        # Sheet 2: Summary by Method
        summary_by_method = df.groupby('Method')[['STOI', 'PESQ', 'MSE', 'MRE']].agg(['mean', 'std']).round(4)
        summary_by_method.to_excel(writer, sheet_name='Summary_By_Method')
        
        # Sheet 3: Summary by Noise Type
        summary_by_noise = df.groupby('Noise_Type')[['STOI', 'PESQ', 'MSE', 'MRE']].agg(['mean', 'std']).round(4)
        summary_by_noise.to_excel(writer, sheet_name='Summary_By_Noise')
    
    # ============== DISPLAY RESULTS TABLE ==============
    print(f"\n{'='*140}")
    print(f"{'Method':<15} {'Noise/DSP':<25} {'SNR_dB':<10} {'STOI':<10} {'PESQ':<10} {'MSE':<12} {'MRE':<12} {'Time(s)':<10}")
    print(f"{'='*140}")
    for r in results_summary:
        print(f"{r['Method']:<15} {r['Noise_Type']:<25} {r['SNR_Input_dB']:<10.4f} {r['STOI']:<10.4f} {r['PESQ']:<10.4f} {r['MSE']:<12.4f} {r['MRE']:<12.4f} {r['Processing_Time_s']:<10.4f}")
    print(f"{'='*140}")
    
    # ============== SUMMARY STATISTICS ==============
    print(f"\n📊 SUMMARY BY METHOD:")
    print(summary_by_method.to_string())
    
    print(f"\n📊 SUMMARY BY NOISE TYPE:")
    print(summary_by_noise.to_string())
    
    print(f"\n✅ Batch processing completed!")
    print(f"   Total experiments: {len(results_summary)}")
    print(f"   Excel file: {excel_file}")
    print(f"   Audio files: results/audio/ ({len(results_summary)} files)")
    print(f"\n📋 Excel Contents:")
    print(f"   • Sheet 1: All_Results ({len(df)} rows × 9 columns)")
    print(f"   • Sheet 2: Summary_By_Method (mean ± std per method)")
    print(f"   • Sheet 3: Summary_By_Noise (mean ± std per noise type)")
    
else:
    print("⏸️  Batch mode disabled.")
    print("   Set run_batch=True to execute comprehensive batch processing.")
    print()
    print("📋 Batch will process:")
    print("   • Deterministic: 5 SNR levels (requires uploaded noise)")
    print("   • Stochastic: 4 noise types × 5 SNR = 20 configs")
    print("   • Traditional: 2 methods × 5 SNR = 10 configs")
    if audio_noise_uploaded is not None:
        print(f"   • Total: 35 experiments")
    else:
        print(f"   • Total: 30 experiments (no deterministic - noise not uploaded)")

## 📥 Download Results


In [ ]:
# In Google Colab, files will be downloaded directly to your browser
# In local Jupyter, files will be accessible in the results/ folder

# Download processed audio files
try:
    from google.colab import files
    print("📥 Downloading processed audio files...")
    
    # Single run results
    for output in os.listdir('outputs'):
        if output.endswith('.wav'):
            files.download(f'outputs/{output}')
            print(f"✅ Downloaded: {output}")
    
    # Batch run results (if exists)
    if os.path.exists('results/batch_results.xlsx'):
        files.download('results/batch_results.xlsx')
        print("✅ Downloaded: batch_results.xlsx")
    
    # Download spectrograms (if exists)
    if os.path.exists('static/spectrograms'):
        for spec in os.listdir('static/spectrograms'):
            if spec.endswith('.png'):
                files.download(f'static/spectrograms/{spec}')
                print(f"✅ Downloaded: {spec}")
    
    print("\n🎉 All files downloaded successfully!")
    
except ImportError:
    print("🗂️ Running locally - results available in:")
    print("   • outputs/ - Processed audio files")
    print("   • results/ - Batch results Excel")
    print("   • static/spectrograms/ - Spectrogram images")


## 📚 Reference & Configuration Details

### **Configuration (matches app.py):**
- **Sample Rate**: 16kHz (fixed for DTLN)
- **Block Length**: 512
- **Block Shift**: 128
- **Colormap**: magma (gamma style, not green/viridis)
- **Spectrogram**: 2x2 combined (Clean, Noise, Mixed, Processed)

### **Processing Methods:**
| Method | Description | Use Case |
|--------|-----------|----------|
| **Deterministic** | Fixed noise + DTLN, constant SNR | Reproducible baseline, requires uploaded noise |
| **Stochastic** | Random noise + DTLN, varying SNR | Robustness testing, synthetic noise |
| **Traditional** | Spectral Sub / Wiener Filter | DSP baseline, no neural network |

### **Quality Metrics (4 decimal precision):**
| Metric | Excellent | Good | Poor | Note |
|--------|-----------|------|------|------|
| **STOI** | >0.8000 | 0.6000-0.8000 | <0.6000 | Speech intelligibility (0-1) |
| **PESQ** | >3.5000 | 2.5000-3.5000 | <2.5000 | Perceptual quality (-0.5 to 4.5) |
| **MSE** | <0.0100 | 0.0100-0.0500 | >0.0500 | Mean squared error (lower better) |
| **MRE** | <0.1000 | 0.1000-0.3000 | >0.3000 | Mean relative error (lower better) |

### **Excel Output (3 Sheets):**
1. **All_Results**: Complete data with 9 columns
   - Method, Noise_Type, SNR_Input_dB
   - STOI, PESQ, MSE, MRE
   - Processing_Time_s, Realtime_Factor
   
2. **Summary_By_Method**: Statistics per method
   - Mean & Standard Deviation
   - Deterministic vs Stochastic vs Traditional
   
3. **Summary_By_Noise**: Statistics per noise type
   - Mean & Standard Deviation
   - Gaussian vs White vs Other

### **Best Practices:**
1. ✅ Start with single run for validation
2. ✅ Use deterministic for reproducible results
3. ✅ Use stochastic for robustness testing
4. ✅ Traditional method good for baseline comparison
5. ✅ DTLN better for speech enhancement
6. ✅ Batch mode for comprehensive statistical analysis

### **File Structure:**
```
outputs/               # Processed audio files (.wav)
results/
  ├── audio/          # Batch processed audio
  ├── spectrograms/   # Combined 2x2 spectrogram images
  ├── metrics/        # Individual metrics (.txt, .png)
  └── batch_results.xlsx  # Excel with 3 sheets
```

### **Web App Alternative:**
For interactive GUI with Tailwind + shadcn/ui design:
```bash
python app.py
```
Open: http://localhost:5000

Web app features:
- 🎨 Modern responsive UI
- 📊 Real-time processing log matrix
- 📥 Excel export from browser
- 🎵 Built-in audio player
- 📉 Combined spectrogram display

---

**Created for DTLN Multi-Method Audio Evaluation Project**  
Compatible with Flask app (app.py) and standalone notebook execution